In [3]:
# import libraries
import pandas as pd
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [5]:
# set (local) path
path = '/Users/KevinBu/Desktop/clemente_lab/Projects/ampaim/'

In [154]:
# load RB metadata
df_RB = pd.read_csv(path + 'inputs/RBB_AMPAIM_MSQ141.csv', header=None)

# remove extraenous info column and notes etc.
df_RB = df_RB.iloc[:26,:10]

# grab names of metadata vars
header_RB = df_RB.iloc[0,:]

# remove descriptive row
df_RB = df_RB.iloc[2:,:]

# header RB is Participant ID, Sex, Race, Hispanic? (Y/N), Age at visit, Disease duration, Disease active?, Diagnosis, ever treated, medications
df_RB.head()

,0,1,2,3,4,5,6,7,8,9
2,703.006.stamp,F,White,N,60,1,NaN,Non-Sjogren's sicca*,N,NaN
3,702.006.stamp,F,White,N,38,1,NaN,Non-Sjogren's sicca*,N,NaN
4,703.005.stamp,F,White,N,44,16,NaN,Non-Sjogren's sicca*,N,NaN
5,403.001.lockit,F,Asian,N,32,1,Y,SLE,Y,HCQ
6,501.002.aimra,F,White,N,82,0.5,Y,RA,Y,MTX


In [174]:
# specimen file 'Subjects.1' has the names which is column 16
df_specimen = pd.read_csv(path + 'inputs/Guma_AMPAIM_subject.tsv', sep='\t', header=None)

# pull the header for later merge
header = df_specimen.iloc[:5,:]

#df_specimen.head(6)
df_specimen = df_specimen.iloc[5:,:]

# df_RB is 24 values, df_specimen is 188, their overlap is 22
# this is in df_RB missing from df_specimen {'601.003.ellipss', '601.001.ellipss'}
# print(set(df_RB.index.values).difference(set(df_specimen[16].values)))
df_specimen = df_specimen.loc[df_specimen[16].isin(list(df_RB[0].values))]

# join on 16 in df_specimen, Participant ID in df_rb
df_specimen = pd.merge(df_specimen, df_RB, how='left', left_on=16, right_on=0)

# add new metadata columns
# additional metadata looks like this 
admd = ['AdditionalMetaData', '__name__', 'Optional', 'Text', 'Limit 256 Characters']

RB_md = ['ParticipantID_RB','Gender_RB', 'Race_RB', 'Hispanic', 'AgeAtVisit', 'DiseaseDuration', 'DiseaseActive', 'Diagnosis', 'EverTreated','Medication']

for x in RB_md:
    admd[1] = x
    header[len(header.columns)] = admd

# rename df_specimen columns to header columns in prep for vertical join
df_specimen.columns = header.columns

# add header to top of df_specimen
df_specimen = pd.concat([header, df_specimen])

# convert None in column 31 Medication to NONE
df_specimen[31] = df_specimen[31].fillna('NoMed')

# export to csv
df_specimen.to_csv(path + 'inputs/df_specimen_MSQ141_KB.tsv', sep='\t', index=False, header=False, na_rep='NA')
df_specimen.head()


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,Ethnicity,Genotypes,Heights,Heights,ICDCode,Illness,Illness,Illness,Illness,Intervention,...,AdditionalMetaData,AdditionalMetaData,AdditionalMetaData,AdditionalMetaData,AdditionalMetaData,AdditionalMetaData,AdditionalMetaData,AdditionalMetaData,AdditionalMetaData,AdditionalMetaData
1,Ethnicity,Genotype,Height,HeightDateCollected,ICDCode,IllnessNotes,IllnessEndDate,IllnessInstanceID,IllnessStartDate,InterventionNotes,...,ParticipantID_RB,Gender_RB,Race_RB,Hispanic,AgeAtVisit,DiseaseDuration,DiseaseActive,Diagnosis,EverTreated,Medication
2,Optional,Optional,Optional,Optional,Optional,Optional,Optional,Optional,Optional,Optional,...,Optional,Optional,Optional,Optional,Optional,Optional,Optional,Optional,Optional,Optional
3,Text,Text,Number,Date,Text,Text,Date,Date,Date,Text,...,Text,Text,Text,Text,Text,Text,Text,Text,Text,Text
4,Limit 45 Characters,Limit 180 Characters,Meters,Year-Month-Day,Limit 8 Characters,Limit 256 Characters,Year-Month-Day,Limit 256 Characters,Year-Month-Day,Limit 256 characters,...,Limit 256 Characters,Limit 256 Characters,Limit 256 Characters,Limit 256 Characters,Limit 256 Characters,Limit 256 Characters,Limit 256 Characters,Limit 256 Characters,Limit 256 Characters,Limit 256 Characters


In [176]:
df_specimen[31]

0             AdditionalMetaData
1                     Medication
2                       Optional
3                           Text
4           Limit 256 Characters
0                          NoMed
1                          NoMed
2                          NoMed
3                            HCQ
4                            MTX
5                          NoMed
6                 HCQ_prednisone
7                          NoMed
8                          NoMed
9                            HCQ
10                         NoMed
11                       MTX_HCQ
12                    prednisone
13                         NoMed
14                      Topicals
15                           HCQ
16                         NoMed
17    Prednisone_MTX_Leflunomide
18                         NoMed
19                           HCQ
20                         NoMed
21                      Topicals
Name: 31, dtype: object

In [148]:
header_RB

0      Participant ID
1                 Sex
2                Race
3     Hispanic? (Y/N)
4        Age at visit
5    Disease duration
6     Disease active?
7           Diagnosis
8       Ever treated 
Name: 0, dtype: object